In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
from torchvision import transforms
from torchvision.datasets.folder import default_loader, has_file_allowed_extension
from torch.utils.data import Dataset
import torch.utils.data as torch_data


class CifarDataLoader:
    """
    For loading Validation data from the ImageNet dataset.
    """

    def __init__(self, image_size: int=32, batch_size: int = 128,
                 is_training: bool = False, num_workers: int = 2, num_samples_per_class: int = None):

        self.batch_size = batch_size

        transform = transforms.Compose([
            # transforms.RandomHorizontalFlip(),
            # transforms.RandomCrop(32, padding=4),
            transforms.Resize((32,32)),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616))
        ])
        if is_training:
            trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
            self._data_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
        else:
            testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
            self._data_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

    @property
    def data_loader(self) -> torch_data.DataLoader:
        """
        Returns the data-loader
        """
        return self._data_loader
    
    def __len__(self):
        return len(self.data_loader)

    def __iter__(self):
        return iter(self.data_loader)

In [5]:
import logging

from tqdm import tqdm
import torch
from torch import nn

# from Examples.torch.utils.image_net_data_loader import ImageNetDataLoader

logger = logging.getLogger('Eval')


class CifarEvaluator:

    def __init__(self):
        self._val_data_loader = CifarDataLoader(is_training=False).data_loader

    def evaluate(self, model: nn.Module, iterations: int = None, use_cuda: bool = False) -> float:
        """
        Evaluate the specified model using the specified number of samples batches from the
        validation set.
        :param model: The model to be evaluated.
        :param iterations: The number of batches to use from the validation set.
        :param use_cuda: If True then use a GPU for inference.
        :return: The accuracy for the sample with the maximum accuracy.
        """

        device = torch.device('cpu')
        if use_cuda:
            if torch.cuda.is_available():
                "here"
                device = torch.device('cuda')
            else:
                logger.error('use_cuda is selected but no cuda device found.')
                raise RuntimeError("Found no CUDA Device while use_cuda is selected")

        if iterations is None:
            logger.info('No value of iteration is provided, running evaluation on complete dataset.')
            iterations = len(self._val_data_loader)
        if iterations <= 0:
            logger.error('Cannot evaluate on %d iterations', iterations)

        correct = 0
        total = 0
        logger.info("Evaluating nn.Module for %d iterations with batch_size %d",
                    iterations, self._val_data_loader.batch_size)

        model = model.to(device)
        model = model.eval()

        with torch.no_grad():
            for i, (input_data, target_data) in tqdm(enumerate(self._val_data_loader), total=iterations):
                if i == iterations:
                    break

                inputs_batch = input_data.to(device)
                target_batch = target_data.to(device)
                predicted_batch = model(inputs_batch)
                _, predicted = predicted_batch.max(1)
                total += target_batch.size(0)
                correct += predicted.eq(target_batch).sum().item()

        return 100 * correct / total


In [6]:
import logging

from tqdm import tqdm
import torch
from torch import nn, optim

logger = logging.getLogger('Trainer')


class CifarTrainer:
    """
    For training a model using the ImageNet dataset.
    """

    # pylint: disable=too-many-arguments
    def __init__(self):
        """
        :param images_dir: The path to the data directory
        :param image_size: The length of the image
        :param batch_size: The batch size to use for training and validation
        :param num_workers: Indiicates to the data loader how many sub-processes to use for data loading.
        :param num_train_samples_per_class: Number of samples to use per class.
        """

        self._train_loader = CifarDataLoader(is_training=True).data_loader

        self._evaluator = CifarEvaluator()

    def _train_loop(self, model: nn.Module, criterion: torch.nn.modules.loss, optimizer: torch.optim,
                    max_iterations: int, current_epoch: int, max_epochs: int,
                    debug_steps: int = 1000, use_cuda: bool = False):
        """
        Train the specified model using the ImageNet dataset for one epoch.
        :param model: The model to train.
        :param criterion: loss function to optimize
        :param optimizer: optimizer function
        :param max_iterations: total number of iterations to train
        :param current_epoch: current epoch for model training, used for reporting logs in debug steps
        :param max_epochs: total epoch for model training, used for reporting logs in debug steps
        :param debug_steps: number of training iterations to report accuracy/loss metrics
        :param use_cuda: If True then use GPU device

        :return: None
        """
        # pylint: disable-msg=too-many-locals
        device = torch.device('cpu')
        if use_cuda:
            if torch.cuda.is_available():
                device = torch.device('cuda')
            else:
                logger.error('use_cuda is selected but no cuda device found.')
                raise RuntimeError("Found no CUDA Device while use_cuda is selected")

        # switch to training mode
        model = model.to(device)
        model.train()

        avg_loss = 0.0

        for i, (images, target) in tqdm(enumerate(self._train_loader), total=max_iterations):
            if i == max_iterations:
                break

            images = images.to(device)
            target = target.to(device)

            # compute model output
            output = model(images)
            loss = criterion(output, target)
            avg_loss += loss.item()

            # compute gradient and do SGD step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % debug_steps == 0:
                eval_accuracy = self._evaluator.evaluate(model, use_cuda=use_cuda)
                logger.info('Epoch #%d/%d: iteration #%d/%d: Global Avg Loss=%f, Eval Accuracy=%f',
                            current_epoch, max_epochs, i, max_iterations,
                            avg_loss / i, eval_accuracy)
                # switch to training mode after evaluation
                model.train()

        eval_accuracy = self._evaluator.evaluate(model, use_cuda=use_cuda)
        print("eval : ", eval_accuracy)
        logger.info('At the end of Epoch #%d/%d: Global Avg Loss=%f, Eval Accuracy=%f',
                    current_epoch, max_epochs, avg_loss / max_iterations, eval_accuracy)

    def train(self, model: nn.Module, max_epochs: int = 20, learning_rate: int = 0.1,
              weight_decay: float = 1e-4, decay_rate: float = 0.1, learning_rate_schedule: list = None,
              debug_steps: int = 1000, use_cuda: bool = False):
        """
        Train the specified model using the ImageNet dataset.
        :param model: The model to train.
        :param max_epochs: Maximum number of epochs to use in training.
        :param learning_rate: Learning rate
        :param weight_decay: Weight decay
        :param decay_rate: Multiplicative factor of learning rate decay
        :param learning_rate_schedule: Adjust the learning rate based on the number of epochs
        :param debug_steps: number of training iterations to report accuracy/loss metrics
        :param use_cuda: If True then use GPU device

        :return: None
        """
        max_iterations = len(self._train_loader)

        loss = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=weight_decay)

        if learning_rate_schedule:
            learning_rate_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, learning_rate_schedule,
                                                                     gamma=decay_rate)

        for current_epoch in range(max_epochs):
            self._train_loop(model, loss, optimizer, max_iterations, current_epoch + 1, max_epochs, debug_steps,
                             use_cuda)

            if learning_rate_schedule:
                learning_rate_scheduler.step()


In [7]:
import os
import torch
from typing import List

class CifarDataPipeline:

    @staticmethod
    def get_val_dataloader() -> torch.utils.data.DataLoader:

        data_loader = CifarDataLoader()
        return data_loader

    @staticmethod
    def evaluate(model: torch.nn.Module, iterations: int, use_cuda: bool) -> float:
        """
        Given a torch model, evaluates its Top-1 accuracy on the dataset
        :param model: the model to evaluate
        :param iterations: the number of batches to be used to evaluate the model. A value of 'None' means the model will be
                           evaluated on the entire dataset once.
        :param use_cuda: whether or not the GPU should be used.
        """
        evaluator = CifarEvaluator()

        return evaluator.evaluate(model, iterations=iterations, use_cuda=use_cuda)

    @staticmethod
    def finetune(model: torch.nn.Module, epochs: int, learning_rate: float, learning_rate_schedule: List, use_cuda: bool):
        """
        Given a torch model, finetunes the model to improve its accuracy
        :param model: the model to finetune
        :param epochs: The number of epochs used during the finetuning step.
        :param learning_rate: The learning rate used during the finetuning step.
        :param learning_rate_schedule: The learning rate schedule used during the finetuning step.
        :param use_cuda: whether or not the GPU should be used.
        """
        trainer = CifarTrainer()

        trainer.train(model, max_epochs=epochs, learning_rate=learning_rate,
                      learning_rate_schedule=learning_rate_schedule, use_cuda=use_cuda)

In [11]:
from resnet import resnet18
model = resnet18(pretrained=True)

In [12]:
accuracy = CifarDataPipeline.evaluate(model, iterations=None, use_cuda=True)
print(accuracy)

100%|██████████| 170498071/170498071 [00:16<00:00, 10323568.46it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


100%|██████████| 79/79 [00:01<00:00, 55.53it/s]

93.07


In [13]:
from decimal import Decimal
from aimet_torch.defs import GreedySelectionParameters, ChannelPruningParameters
from aimet_common.defs import CompressionScheme, CostMetric

greedy_params = GreedySelectionParameters(target_comp_ratio=Decimal(0.9),
                                          num_comp_ratio_candidates=3)
modules_to_ignore = [model.conv1]
auto_params = ChannelPruningParameters.AutoModeParams(greedy_select_params=greedy_params,
                                                      modules_to_ignore=modules_to_ignore)
data_loader = CifarDataPipeline.get_val_dataloader()
params = ChannelPruningParameters(data_loader=data_loader,
                                  num_reconstruction_samples=10,
                                  allow_custom_downsample_ops=False,
                                  mode=ChannelPruningParameters.Mode.auto,
                                  params=auto_params)

eval_callback = CifarDataPipeline.evaluate
eval_iterations = 1
compress_scheme = CompressionScheme.channel_pruning
cost_metric = CostMetric.mac

Files already downloaded and verified


In [14]:
from aimet_torch.compress import ModelCompressor
compressed_model, comp_stats = ModelCompressor.compress_model(model=model,
                                                              eval_callback=eval_callback,
                                                              eval_iterations=eval_iterations,
                                                              input_shape=(1, 3, 32, 32),
                                                              compress_scheme=compress_scheme,
                                                              cost_metric=cost_metric,
                                                              parameters=params)

print(comp_stats)

2024-12-23 05:57:10,410 - root - INFO - AIMET


2024-12-23 05:57:15,455 - CompRatioSelect - INFO - Analyzing compression ratio: 0.3333333333333333333333333333 =====================>


/usr/local/lib/python3.10/dist-packages/aimet_torch/winnow/mask_propagation_winnower.py:95: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dummy_input = torch.tensor(dummy_input).cuda()  # pylint: disable=not-callable


2024-12-23 05:57:17,071 - ChannelPruning - INFO - finished linear regression fit 
Files already downloaded and verified
2024-12-23 05:57:17,585 - Trainer - INFO - Evaluating nn.Module for 1 iterations with batch_size 128


100%|██████████| 1/1 [00:00<00:00, 28.51it/s]

...

2024-12-23 05:58:33,668 - ChannelPruning - INFO - finished linear regression fit 
Files already downloaded and verified
2024-12-23 05:58:34,199 - Trainer - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-23 05:58:34,200 - Trainer - INFO - Evaluating nn.Module for 79 iterations with batch_size 128


100%|██████████| 79/79 [00:00<00:00, 87.20it/s] 


Files already downloaded and verified
2024-12-23 05:58:35,692 - Trainer - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-23 05:58:35,693 - Trainer - INFO - Evaluating nn.Module for 79 iterations with batch_size 128


100%|██████████| 79/79 [00:00<00:00, 103.02it/s]

**********************************************************************************************
Compressed Model Statistics
Baseline model accuracy: 93.070000, Compressed model accuracy: 87.530000
Compression ratio for memory=0.991126, mac=0.819081

**********************************************************************************************

Per-layer Stats
    Name:layer1.0.conv1, compression-ratio: None
    Name:layer1.0.conv2, compression-ratio: 0.3333333333333333333333333333
    Name:layer1.1.conv1, compression-ratio: None
    Name:layer1.1.conv2, compression-ratio: 0.3333333333333333333333333333
    Name:layer2.0.conv1, compression-ratio: None
    Name:layer2.0.conv2, compression-ratio: None
    Name:layer2.0.downsample.0, compression-ratio: None
    Name:layer2.1.conv1, compression-ratio: None
    Name:layer2.1.conv2, compression-ratio: None
    Name:layer3.0.conv1, compression-ratio: 0.6666666666666666666666666666
    Name:layer3.0.conv2, compression-ratio: None
    Name:layer3

In [15]:
accuracy = CifarDataPipeline.evaluate(compressed_model, iterations=None, use_cuda=True)
print(accuracy)

Files already downloaded and verified
2024-12-23 05:58:37,040 - Trainer - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-23 05:58:37,040 - Trainer - INFO - Evaluating nn.Module for 79 iterations with batch_size 128


100%|██████████| 79/79 [00:00<00:00, 103.49it/s]

87.53


In [20]:

CifarDataPipeline.finetune(compressed_model, epochs=10, learning_rate=10e-4, learning_rate_schedule=[5, 10],
                              use_cuda=True)

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 391/391 [00:06<00:00, 55.91it/s]

2024-12-23 06:01:04,537 - Trainer - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-23 06:01:04,537 - Trainer - INFO - Evaluating nn.Module for 79 iterations with batch_size 128



100%|██████████| 79/79 [00:00<00:00, 102.69it/s]

eval :  90.33
2024-12-23 06:01:05,410 - Trainer - INFO - At the end of Epoch #1/10: Global Avg Loss=0.013821, Eval Accuracy=90.330000


...

eval :  90.7
2024-12-23 06:02:09,526 - Trainer - INFO - At the end of Epoch #9/10: Global Avg Loss=0.001778, Eval Accuracy=90.700000



100%|██████████| 391/391 [00:07<00:00, 54.83it/s]

2024-12-23 06:02:16,771 - Trainer - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-23 06:02:16,771 - Trainer - INFO - Evaluating nn.Module for 79 iterations with batch_size 128



100%|██████████| 79/79 [00:00<00:00, 102.48it/s]

eval :  90.61
2024-12-23 06:02:17,627 - Trainer - INFO - At the end of Epoch #10/10: Global Avg Loss=0.001707, Eval Accuracy=90.610000


In [21]:
accuracy = CifarDataPipeline.evaluate(compressed_model, iterations=None, use_cuda=True)
print(accuracy)

Files already downloaded and verified
2024-12-23 06:02:21,009 - Trainer - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-23 06:02:21,010 - Trainer - INFO - Evaluating nn.Module for 79 iterations with batch_size 128


100%|██████████| 79/79 [00:00<00:00, 102.30it/s]

90.61


In [ ]:
os.makedirs('./output/', exist_ok=True)
torch.save(compressed_model, './output/finetuned_model')